In [ ]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')
# make compatible with Python 2 and Python 3
from __future__ import print_function, division, absolute_import 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
Hourly = pd.read_csv('~/Desktop/data-x-data/ProcessedData/House1/House1_1h.csv',parse_dates = True,index_col =0)

In [ ]:
Hourly.head(2)

In [ ]:
series = Hourly['Fridge']

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
# Define function to test stationairty of data
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
test_stationarity(series)

In [ ]:
# Calculates ACF & PACF
from statsmodels.tsa.stattools import acf, pacf
lag_acf = acf(series, nlags=20)
lag_pacf = pacf(series, nlags=20, method='ols')

In [ ]:
# Plot ACF
import statsmodels
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.stattools
import statsmodels.graphics.tsaplots
fig_acf = statsmodels.graphics.tsaplots.plot_acf(series, lags=30)

In [ ]:
# Plot PACF
fig_pcaf = statsmodels.graphics.tsaplots.plot_pacf(series, lags=30)

In [ ]:
# Test whether we should difference at the alpha=0.05 significance level
from pyramid.arima.stationarity import ADFTest

adf_test = ADFTest(alpha=0.05)
p_val, should_diff = adf_test.is_stationary(series)
print(p_val, should_diff)

In [ ]:
# Test whether we should difference using differnet criteria
from pyramid.arima.utils import ndiffs

# Estimate the number of differences using an ADF test:
n_adf = ndiffs(series, test='adf')  # -> 0
print(n_adf)
# Or a KPSS test (auto_arima default):
n_kpss = ndiffs(series, test='kpss')  # -> 0
print(n_kpss)
# Or a PP test:
n_pp = ndiffs(series, test='pp')  # -> 0
print(n_pp)

#assert n_adf == n_kpss == n_pp == 0

In [ ]:
# Build Arima model on Fridge energy consumption, test on (p,d,q) orders
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(series, order=(4,0,3))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

In [ ]:
# Build rolling arima model & make predictions
X = series.values
size = int(len(X) * 0.98)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(4,0,3))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)


# calculate mean absolute error
mean_absolute_error(test, predictions)

In [ ]:
# plot predicted values and true values against time
plt.rcParams['figure.figsize'] = (15, 6)
fig, ax = plt.subplots()

ax.plot(X[size:len(X)],color='orange', linestyle='-', label='Ture')
ax.plot(predictions, linestyle='-', label = 'Forecast')
ax.set_xlabel('hour') # add xlabel
ax.set_ylabel('Energy') # add xlabel

ax.legend(loc=2) # location of legend is an integer, 

In [ ]:
# Define function to calculate mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

In [ ]:
# Calculate mean absolute percentage error
forecast = np.concatenate(predictions, axis=0)
mean_absolute_percentage_error(test, forecast)

In [ ]:
# Get a subset of first 500 hours of Fridge energy consumption
small = series[:500]
# Decompose the subset to study components of data
plt.rcParams['figure.figsize'] = (10, 4)
from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(small,model='additive')
fig = result.plot()
# Find seasonal component, seanson = 24 hours(1 day)
